In [1]:
import pandas as pd

df = pd.read_csv('avocado_df.csv', index_col=0)

display(df)

,region,AveragePrice,Total Volume,latitude,longitude,geometry
0,Albany,1.561036,4.753787e+04,42.651550,-73.755211,POINT (-73.75521087646484 42.65155029296875)
1,Atlanta,1.337959,2.621453e+05,33.748310,-84.391113,POINT (-84.39111328125 33.74831008911133)
2,BaltimoreWashington,1.534231,3.985619e+05,39.183739,-76.674210,POINT (-76.67420959472656 39.18373870849609)
3,Boise,1.348136,4.264257e+04,43.607639,-116.193398,POINT (-116.1933975219727 43.60763931274414)
4,Boston,1.530888,2.877929e+05,42.358662,-71.056740,POINT (-71.05673980712891 42.35866165161133)
5,BuffaloRochester,1.516834,6.793630e+04,42.885441,-78.878464,POINT (-78.87846374511719 42.88544082641602)
6,California,1.395325,3.044324e+06,37.255100,-119.617523,POINT (-119.6175231933594 37.25510025024414)
7,Charlotte,1.606036,1.051939e+05,35.222858,-80.837959,POINT (-80.83795928955078 35.22285842895508)
8,Chicago,1.556775,3.955690e+05,41.884251,-87.632446,POINT (-87.6324462890625 41.88425064086914)
9,CincinnatiDayton,1.209201,1.317219e+05,39.107128,-84.504127,POINT (-84.50412750244141 39.10712814331055)


In [3]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure, ColumnDataSource
from bokeh.tile_providers import get_provider, CARTODBPOSITRON
from bokeh.palettes import PRGn, RdYlGn
from bokeh.transform import linear_cmap,factor_cmap
from bokeh.layouts import row, column
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter

import numpy as np


# Define function to switch from lat/long to mercator coordinates
def x_coord(x, y):
    
    lat = x
    lon = y
    
    r_major = 6378137.000
    x = r_major * np.radians(lon)
    scale = x/lon
    y = 180.0/np.pi * np.log(np.tan(np.pi/4.0 + 
        lat * (np.pi/180.0)/2.0)) * scale
    return (x, y)

# Define coord as tuple (lat,long)
df['coord'] = list(zip(df['latitude'], df['longitude']))


# Obtain list of mercator coordinates
mercators = [x_coord(x, y) for x, y in df['coord'] ]


In [4]:

# Create column mercator
df['mercator'] = mercators

# Define columns mercator_x and mercator_y
df[['mercator_x', 'mercator_y']] = df['mercator'].apply(pd.Series)

# Drop 'geometry' column 
df = df.drop(columns=['geometry'])

# Preview
df.head()


,region,AveragePrice,Total Volume,latitude,longitude,coord,mercator,mercator_x,mercator_y
0,Albany,1.561036,47537.86973,42.651550,-73.755211,"(42.65155029, -73.75521088)","(-8210392.518512112, 5259083.750058974)",-8.210393e+06,5.259084e+06
1,Atlanta,1.337959,262145.32200,33.748310,-84.391113,"(33.74831009, -84.39111328)","(-9394375.757807067, 3995056.081948725)",-9.394376e+06,3.995056e+06
2,BaltimoreWashington,1.534231,398561.89150,39.183739,-76.674210,"(39.18373871, -76.67420959)","(-8535333.968535533, 4748024.844048325)",-8.535334e+06,4.748025e+06
3,Boise,1.348136,42642.56731,43.607639,-116.193398,"(43.60763931, -116.1933975)","(-12934589.843240427, 5404922.878631644)",-1.293459e+07,5.404923e+06
4,Boston,1.530888,287792.85450,42.358662,-71.056740,"(42.35866165, -71.05673981)","(-7910000.093079331, 5214857.553783183)",-7.910000e+06,5.214858e+06


In [5]:
# Create map with Bokeh

# Set underlying map
chosentile = get_provider(CARTODBPOSITRON)

# Set palette
palette = PRGn[11]

# Set source to df
source = ColumnDataSource(data=df)

# Define color mapper
color_mapper = linear_cmap(field_name = 'AveragePrice', palette = palette, low = df['AveragePrice'].min(), high = df['AveragePrice'].max())

# Set tooltips
nan_color = '#d9d9d9'
tooltips = [("Price","@AveragePrice"), ("Region","@region")]


# Create figure
p = figure(title = 'Avocado Prices by region in the United States', x_axis_type="mercator", y_axis_type="mercator", 
           x_axis_label = 'Longitude', y_axis_label = 'Latitude', tooltips = tooltips)

# Add map tile
p.add_tile(chosentile)

# Add points using mercator coordinates
p.circle(x = 'mercator_x', y = 'mercator_y', color = color_mapper, source=source, size=30, fill_alpha = 0.7)

#Defines color bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], 
                     formatter = NumeralTickFormatter(format='0.0[0000]'), 
                     label_standoff = 13, width=8, location=(0,0))

# Set color_bar location
p.add_layout(color_bar, 'right')

# Display in notebook
output_notebook()

# Save as HTML
output_file('avocado.html', title='Avocado Prices by region in the United States')

# Show map
show(p)



Loading BokehJS ...